### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account

# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.sales-salesterritory"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela com substituição de '-' por '_'
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    # Ler os dados da tabela do BigQuery para um df
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    # Armazenar o df no dicionário
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
sales_salesterritory = df_processados['sales_salesterritory']

print(f"Colunas: {sales_salesterritory.shape[1]}\nLinhas: {sales_salesterritory.shape[0]}")

In [ ]:
# Identificar duplicadas com base em 'territoryid'
duplicadas = sales_salesterritory[
    sales_salesterritory.duplicated(subset=['territoryid'], keep=False)
]

# Verificar se existem duplicadas
if not duplicadas.empty:
    # Ordenar duplicadas por 'territoryid' e 'modifieddate'
    duplicadas_ordenadas = duplicadas.sort_values(by=['territoryid', 'modifieddate'])

    # Exibir duplicadas ordenadas
    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

In [ ]:
# Remover duplicadas mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
# Importante, pois se houver erro na ingestão (duplicação), mantém os dados integros.

sales_salesterritory = sales_salesterritory.drop_duplicates(subset=['territoryid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(sales_salesterritory)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = sales_salesterritory.copy()


#doc: bkp = backup (cópia)

In [ ]:
# Ordenar e exibir o df por 'territoryid'
sales_salesterritory = sales_salesterritory.sort_values(by=['territoryid'])

print(sales_salesterritory)

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in sales_salesterritory.columns:   
    missing_rows = sales_salesterritory[sales_salesterritory[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = sales_salesterritory.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

#doc: currentflag possue somente 1 valor, o que indica que pode ser somente valores True ou False.

In [ ]:
# Identificar colunas com apenas 1 valor único (incluindo NaN)
colunas_com_1_valor = valores_unicos[valores_unicos == 1].index

# Exibir os valores únicos dessas colunas
print("Colunas com apenas 1 valor único:")
for col in colunas_com_1_valor:
    valor_unico = sales_salesterritory[col].unique()
    print(f"{col}: {valor_unico}")


In [ ]:
# Identificar colunas com valor único igual a 0
colunas_para_remover = [col for col in sales_salesterritory.columns if sales_salesterritory[col].nunique() == 1 and sales_salesterritory[col].unique()[0] == 0]

# Remover as colunas identificadas
sales_salesterritory = sales_salesterritory.drop(columns=colunas_para_remover)

# Exibir o DataFrame atualizado e as colunas restantes
print("Colunas restantes no DataFrame:")
print(sales_salesterritory.columns.tolist())

# Visualizar os dados atualizados
print("\nDataFrame atualizado:")
print(sales_salesterritory)


#doc*: as colunas costytf e Length apresentaram todas as linhas com o valor 0, portanto, sem valor agregado as análises - colunas deletadas
#      Mantive a coluna modifieddate, por se tratar de datas e ser fundamental em análises futuras

In [ ]:
sales_salesterritory.info()

In [ ]:
#avaliando as variáveis qualitativas*

coluna_quantitativa = ["name", "countryregioncode", "group"]
for col in coluna_quantitativa:
    counts = sales_salesterritory[col].value_counts().nlargest(10)
    percentages = (counts / sales_salesterritory.shape[0] * 100).map("{:.2f}%".format)
    summary = pd.DataFrame({"qtde.": counts, "%": percentages})
    print(summary)    


#doc*: variáveis qualitativas são um tipo de variável estatística que representam características ou atributos dos dados, sem serem medidas numericamente
#      no nosso caso, name, countryregioncode, group, por exemplo

In [ ]:
#dominância geográfica*

group_counts = sales_salesterritory['group'].value_counts()


sns.set(style='ticks', rc={"axes.facecolor": "black", "figure.facecolor": "black"})
plt.figure(figsize=(8, 5))
ax = sns.barplot(x=group_counts.index, y=group_counts.values, palette='viridis')
plt.title("Distribuição de Grupos", color='white', fontsize=14)
plt.xlabel("Grupo", color='white', fontsize=12)
plt.ylabel("", fontsize=12)  
plt.xticks(color='white', fontsize=10)
ax.tick_params(axis='y', left=False, labelleft=False)
ax.grid(False)

for container in ax.containers:
    ax.bar_label(container, fmt='%d', color='white', fontsize=10, label_type='edge')

plt.show()







#doc*: a região North America domina com 60% das ocorrências no grupo,
#      o código de país US representa 50% dos dados e CA repsenta 10%  

In [ ]:
# alinhamento de grupos com países*

group_country = sales_salesterritory.groupby(['group', 'countryregioncode']).size().unstack(fill_value=0)

# Heatmap para visualização
plt.figure(figsize=(10, 6))
sns.heatmap(group_country, annot=True, fmt='d', cmap='coolwarm', cbar=False)
plt.title("Alinhamento de Grupos com Países", color='white', fontsize=14)
plt.xlabel("Código de País", color='white', fontsize=12)
plt.ylabel("Grupo", color='white', fontsize=12)
plt.xticks(color='white', fontsize=10)
plt.yticks(color='white', fontsize=10)

plt.show()


#doc*: identificar quais países pertencem a quais grupos e se há algum desequilíbrio na distribuição (North America: US=5;CA=1)
#      entender a relação entre grupos e países ajuda a otimizar operações regionais, ajustar estratégias de marketing e identificar áreas de oportunidades de crescimento.
#      grupos sub-representados em países específicos podem indicar mercados a serem explorados

In [ ]:
# Desempenho de vendas por grupo*

#soma salesytd e saleslastyear**
group_performance = sales_salesterritory.groupby('group')[['salesytd', 'saleslastyear']].sum()

fig, ax = plt.subplots(figsize=(12, 6))
group_performance.plot(kind='bar', color=['blue', 'green'], edgecolor='black', ax=ax)
plt.title("Desempenho de Vendas por Grupo", color='white', fontsize=14)
plt.xlabel("Grupo", color='white', fontsize=12)
plt.ylabel("Vendas", color='white', fontsize=12)
plt.xticks(color='white', fontsize=10, rotation=0)
ax.yaxis.set_visible(False)
plt.legend(['Vendas YTD', 'Vendas Ano Passado'], fontsize=10, loc='upper left')

for container in ax.containers:
    labels = [f'{value / 1_000_000:.2f}M' for value in container.datavalues]
    ax.bar_label(container, labels=labels, label_type='edge', color='white', fontsize=10)

plt.show()







#doc*: comparar o desempenho de vendas entre os diferentes grupos com base nas métricas "salesytd" (vendas acumuladas no ano) e "saleslastyear" (vendas do ano passado)
#      grupos com desempenho consistente indicam estabilidade e podem ser mantidos
#      grupos com queda nas vendas requerem atenção para identificar e corrigir problemas


#doc: vendas YTD (Year-to-Date): representa o total acumulado de vendas desde o início do ano até a data atual.Útil para avaliar o desempenho atual em relação ao ano anterior
#       vendas do Ano Passado (Last Year): refere-se ao total de vendas realizadas durante o ano anterior completo. Comparação para identificar tendências de crescimento ou declínio ao longo do tempo


#doc: as colunas 'salesytd' e 'saleslastyear' não são calculadas dinamicamente,elas representam valores acumulados registrados na base de dados até a data 
#     informada em 'modifieddate', que é 2008-04-30 para todos os registros

In [ ]:
# Receita total por país*
country_performance = sales_salesterritory.groupby('countryregioncode')['salesytd'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
ax = sns.barplot(x=country_performance.index, y=country_performance.values, palette='mako')
plt.title("Receita Total por País", color='white', fontsize=14)
plt.xlabel("Código de País", color='white', fontsize=12)
plt.ylabel("", fontsize=12)  
plt.xticks(color='white', fontsize=10)
ax.tick_params(axis='y', left=False, labelleft=False)

for container in ax.containers:
    labels = [f'{value / 1_000_000:.2f}M' for value in container.datavalues]
    ax.bar_label(container, labels=labels, label_type='edge', color='white', fontsize=10)
    
plt.show()



# doc*:representação da receita total (SalesYTD) acumulada por país, destacando quais nações geraram mais receita,
#      ajuda a entender a distribuição geográfica das vendas e a identificar mercados mais lucrativos para a empresa

# doc: países com maior receita, como os no topo do gráfico, são os principais contribuintes para o faturamento,
#      comparação direta entre países auxilia na análise de desempenho regional e na alocação de recursos

#doc: a coluna 'salesytd' não é calculada dinamicamente,ela representa valores acumulados registrados na base de dados até a data 
#     informada em 'modifieddate', que é 2008-04-30 para todos os registros

In [ ]:
sales_salesterritory.info()

In [ ]:
# Variáveis quantitativas*: estatísticas descritivas para verificar se ainda há o que ser feito antes de exportar os dados ao BigQuery

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['salesytd', 'saleslastyear']

# Estatísticas Descritivas das colunas numéricas*
print(sales_salesterritory[numeric_columns].describe())

# Cálculo de limites para outliers (IQR)**
for col in numeric_columns:
    q1 = sales_salesterritory[col].quantile(0.25)
    q3 = sales_salesterritory[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Detecção e Análise de Outliers***
    outliers = sales_salesterritory[(sales_salesterritory[col] < lower_bound) | (sales_salesterritory[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


#doc*: variáveis quantitativas são um tipo de dado que pode ser representado por números e medidas objetivas, no nosso caso, vacationhours, sickleavehours, por exemplo
#doc*: realizar estatísticas descritivas para entender a centralidade e variação dos dados (valores médios, mínimos, máximos, etc.)
#doc**: calcular limites para identificar outliers (valores extremos que podem indicar erros ou casos excepcionais nos dados)
#doc***: verificar a existência de outliers para decidir ações como remoção, substituição ou tratamento, garantindo qualidade dos dados
#doc****: as colunas analisadas não apresentam outliers, pois os dados estão dentro dos limites esperados, sugerindo que não há necessidade de tratamento adicional para valores extremos,
#         isso indica boa qualidade dos dados para essas variáveis e que elas estão prontas para serem exportadas ou utilizadas em análises e modelos

In [ ]:
# Configurar o estilo do gráfico para fundo escuro
sns.set(style='darkgrid', rc={"axes.facecolor": "black", "figure.facecolor": "black"})

# Lista das colunas para análise
columns_to_plot = ['salesytd', 'saleslastyear']

# Dividir os valores por 1.000.000 para representar em milhões
sales_salesterritory[columns_to_plot] = sales_salesterritory[columns_to_plot] / 1_000_000

# Configurar a grade para 2 gráficos, um embaixo do outro
fig, axes = plt.subplots(len(columns_to_plot), 1, figsize=(10, 12))

for ax, col in zip(axes, columns_to_plot):
    quartiles = sales_salesterritory.groupby('group')[col].quantile([0.25, 0.50, 0.75]).unstack()
    sns.boxplot(
        x='group', y=col, data=sales_salesterritory, ax=ax, showmeans=True,
        meanprops={"marker": "o", "markerfacecolor": "white", "markeredgecolor": "black", "markersize": 6},
        boxprops={"color": "blue"}, whiskerprops={"color": "blue"}, capprops={"color": "blue"},
        medianprops={"color": "red"}, flierprops={"markerfacecolor": "yellow", "markersize": 6}
    )
    ax.set_title(f'Distribuição de {col} por Grupo (em milhões)', color='white', fontsize=14)
    ax.set_xlabel("Grupo", color='white', fontsize=12)
    ax.set_ylabel("", fontsize=12)  
    ax.tick_params(axis='x', colors='white', labelrotation=45, labelsize=10)
    ax.tick_params(axis='y', colors='white', left=False)  
    ax.grid(False)

    # Adicionar os quartis como texto
    for group, values in quartiles.iterrows():
        if values.notnull().all(): 
            x_pos = list(sales_salesterritory['group'].unique()).index(group)  
            offset = 0.42
            for i, quartile in enumerate(values):
                y_pos = quartile
                ax.text(x=x_pos + offset, y=y_pos, s=f"{quartile:.2f}M", ha='left', va='center', fontsize=10, color='white')

plt.tight_layout()

plt.show()

In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['sales_salesterritory'] = sales_salesterritory

In [ ]:
sales_salesterritory.head()

In [ ]:
# Padronizar colunas com valores textuais
sales_salesterritory['name'] = sales_salesterritory['name'].str.strip().str.upper()
sales_salesterritory['countryregioncode'] = sales_salesterritory['countryregioncode'].str.strip().str.upper()
sales_salesterritory['group'] = sales_salesterritory['group'].str.strip().str.upper()
sales_salesterritory['rowguid'] = sales_salesterritory['rowguid'].str.strip().str.upper()

print(sales_salesterritory.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:

# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
    # Nome da tabela no BigQuery
    output_table = f"{output_dataset}.{table_name}"

    # Configurar job de exportação
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    # Exportar DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")